In [1]:
import trimesh

scene = trimesh.Scene()

In [2]:
from my_code.datasets.smpl_webuser.serialization import load_model
from my_code.datasets.smal_online.my_mesh.mesh import myMesh
import pickle as pkl

# Load the smal model 
model_path = '/home/s94zalek_hpc/shape_matching/my_code/datasets/smal_online/smal_CVPR2017.pkl'
model = load_model(model_path)

In [3]:
# Save the mean model
# model.r are the model vertexes, and model.f are the mesh faces.

mesh_mean = trimesh.Trimesh(vertices=model.r, faces=model.f)

In [4]:
# Load the family clusters data (see paper for details)
# and save the mean per-family shape
# 0-felidae(cats); 1-canidae(dogs); 2-equidae(horses);
# 3-bovidae(cows); 4-hippopotamidae(hippos);
# The clusters are over the shape coefficients (betas);
# setting different betas changes the shape of the model

model_data_path = '/home/s94zalek_hpc/shape_matching/my_code/datasets/smal_online/smal_CVPR2017_data.pkl'
with open(model_data_path, 'rb') as model_data_path:
    data = pkl.load(model_data_path, encoding="latin1")

In [ ]:
import numpy as np

scene.geometry.clear()

for i, betas in enumerate(data['cluster_means']):
    model.betas[:] = betas
    model.pose[:] = 0.
    model.trans[:] = 0.
    # m = myMesh(v=model.r, f=model.f)
    # m.save_ply('family_'+str(i)+'.ply')
    
    mesh_i = trimesh.Trimesh(
        vertices=model.r + np.array([2*i, 0, 0]),
        faces=model.f)
    
    scene.add_geometry(mesh_i)
    
print('saved mean per-family shape')

scene.show()


In [ ]:
scene.geometry.clear()

# Show the toys in T-pose
for i, betas in enumerate(data['toys_betas']):
    model.betas[:] = betas
    model.pose[:] = 0.
    model.trans[:] = 0.
    # m = myMesh(v=model.r, f=model.f)
    # m.save_ply('toy_'+str(i)+'.ply')
    
    mesh_i = trimesh.Trimesh(
        vertices=model.r + np.array([2*i, 0, 0]),
        faces=model.f)
    
    scene.add_geometry(mesh_i)
    
print('saved toys in t-pose')

scene.show()


In [ ]:
data['toys_betas'].shape

In [ ]:
data['cluster_means'].shape

In [ ]:
data['cluster_means'].max()

## Generate data

In [ ]:
betas = data['cluster_means'][4]

model.betas[:] = betas
model.pose[:] = 0.
model.trans[:] = 0.

scene.geometry.clear()

for i in range(10):
    model.pose[0:3]=0
    model.pose[3:]=0.2 * np.random.randn(96)
    
    
    # print(model.betas)
    
    # model.betas[:] = betas + 0.5 * np.random.randn(*betas.shape)
    
    print(model.r)
    
    # print(model.betas)
    
    
    
    mesh_i = trimesh.Trimesh(
        vertices=model.r + np.array([3*i, 0, 0]),
        faces=model.f)
    
    
    scene.add_geometry(mesh_i)

scene.show()
 

In [44]:
import my_code.datasets.preprocessing as preprocessing
import torch


def process_mesh(mesh):

    item = dict()

    item['verts'] = torch.from_numpy(mesh.vertices).float()
    item['faces'] = torch.from_numpy(mesh.faces).long()
            
    # item['verts'] = preprocessing.center_bbox(item['verts'])

    # normalize vertices by area
    item['verts'] = preprocessing.normalize_face_area(item['verts'], item['faces'])


    item = preprocessing.get_spectral_ops(item, num_evecs=32,
                                                    cache_dir=None)
    
    mesh_out = trimesh.Trimesh(vertices=item['verts'].numpy(), faces=item['faces'].numpy())
    
    return item, mesh_out


In [ ]:
betas = data['cluster_means'][0]

model.betas[:] = betas
model.pose[:] = 0.
model.trans[:] = 0.


model.pose[0:3]=0
model.pose[3:]=0.2 * np.random.randn(96)
# model.betas[:] = betas + 0.5 * np.random.randn(*betas.shape)
    
mesh_1_trimesh = trimesh.Trimesh(
    vertices=model.r,
    faces=model.f)

mesh_1, mesh_1_trimesh = process_mesh(mesh_1_trimesh)




betas = data['cluster_means'][0]

model.betas[:] = betas
model.pose[:] = 0.
model.trans[:] = 0.


model.pose[0:3]=0
model.pose[3:]=0.2 * np.random.randn(96)
# model.betas[:] = betas + 0.5 * np.random.randn(*betas.shape)
    
mesh_2_trimesh = trimesh.Trimesh(
    vertices=model.r + np.array([0, 0, 0]),
    faces=model.f)

mesh_2, mesh_2_trimesh = process_mesh(mesh_2_trimesh)   




scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    
    mesh_2['verts'], mesh_2['faces'],
    mesh_1['verts'], mesh_1['faces'],
    
    torch.arange(mesh_1['verts'].shape[0]).int(),
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()

In [ ]:
import my_code.utils.plotting_utils as plotting_utils
import matplotlib.pyplot as plt
import torch


C_gt_xy_lstsq = torch.linalg.lstsq(
    mesh_2['evecs'],
    mesh_1['evecs']
    ).solution

l = 0
h = 32

fig, axs = plt.subplots(1, 1, figsize=(4, 4))

plotting_utils.plot_Cxy(fig, axs,  C_gt_xy_lstsq,
                        'before', l, h, show_grid=False, show_colorbar=False)
plt.show()

In [ ]:
lion_data_path = '/home/s94zalek_hpc/shape_matching/data/SMAL_fitted/big_cats/lion6.pkl'

with open(lion_data_path, 'rb') as lion_data_path:
    lion_data = pkl.load(lion_data_path, encoding="latin1")
    
lion_data

### Generate other poses for each train shape

In [ ]:
train_names = '/home/s94zalek_hpc/shape_matching/data/SMAL_r/train_cat.txt'
# read the train names
with open(train_names) as f:
    train_names = f.readlines()
    train_names = [x.strip() for x in train_names]
    
    
train_names

In [ ]:
# read name map 

name_map = '/home/s94zalek_hpc/shape_matching/data/SMAL_r/name_map.txt'

with open(name_map) as f:
    name_map = f.readlines()
    name_map = [x.strip() for x in name_map]
    
    name_map = dict([x.split()[::-1] for x in name_map])
    
name_map

In [ ]:
# for train_name in train_names:
    

orig_name = name_map['cougar_01']

orig_path = f'/home/s94zalek_hpc/shape_matching/data/SMAL_fitted/{orig_name}.pkl'
with open(orig_path, 'rb') as orig_path:
    shape_data = pkl.load(orig_path, encoding="latin1")
    
    
model.betas[:] = np.zeros_like(model.betas)
model.betas[:20] = np.array(shape_data['beta']) + 0.05 * np.random.randn(*shape_data['beta'].shape)

model.pose[:] = np.array(shape_data['pose']) + 0.2 * np.random.randn(*shape_data['pose'].shape)
model.pose[0:3]=0


mesh_trimesh = trimesh.Trimesh(
    vertices=model.r,
    faces=model.f)

scene.geometry.clear()

scene.add_geometry(mesh_trimesh)


scene.set_camera()
scene.show()


    
   

In [ ]:
mesh_orig = trimesh.load(
    '/home/s94zalek_hpc/shape_matching/data/SMAL_r/off/hippo_03.off'
)
mesh_orig.show()

In [ ]:
shape_data

In [ ]:
chumpy.ch.Ch object

In [ ]:
data['cluster_means'][1]